In [8]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sklearn.model_selection as model_selection
import pandas as pd

In [14]:
# Location des fichiers
DATADIR = "C:\\Users\\Ganthier\\Desktop\\Mango"  
CATEGORIES = ["mango_corn", "mango_fil", "mango_Fransik"]         

In [15]:
# Lecture des fichiers dans numpy array
IMG_SIZE = 50
training_data = []
for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(DATADIR,category)  # create path to dogs and cats
    class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat
    for img in os.listdir(path):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
                #plt.imshow(img_array, cmap='gray')  # graph it
                #plt.show()             
            except Exception as e:  # in the interest in keeping the output clean...
                pass

In [16]:
X = []
y = []
for features,label in training_data:
    X.append(features)
    y.append(label)

#plt.imshow(X[0], cmap='gray')  # graph it
#plt.show() 

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)      # reformatter les features
#print(X.shape)
#print(X.shape[1:])

X = X.astype('float32')
X = X/255.0
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=5)

In [23]:
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, shuffle=True)
keras.models.save_model(model, "C:\\Users\\Ganthier\\Desktop\\img.model")
model.evaluate(X_test, y_test)

Epoch 1/5
1204/1204 [==============================] - 3s 3ms/sample - loss: 0.6595 - acc: 0.7259
Epoch 2/5
1204/1204 [==============================] - 2s 2ms/sample - loss: 0.3449 - acc: 0.8729
Epoch 3/5
1204/1204 [==============================] - 2s 2ms/sample - loss: 0.3095 - acc: 0.8929
Epoch 4/5
1204/1204 [==============================] - 2s 2ms/sample - loss: 0.2240 - acc: 0.9178
Epoch 5/5
301/301 [==============================] - 1s 3ms/sample - loss: 0.1576 - acc: 0.9635


[0.15762375780316287, 0.96345514]

In [24]:
y_test_pred = model.predict(X_test)

test_result = pd.DataFrame(y_test_pred, y_test)
#print(test_result)

In [25]:
def prepare(filepath):
    #IMG_SIZE = 70  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.

prediction= model.predict([prepare("C:\\Users\\Ganthier\\Desktop\\Mango\\mango_corn\\output_image-0001.jpg" )])
prediction1 = model.predict([prepare("C:\\Users\\Ganthier\\Desktop\\Mango\\mango_fil\\output_image0001.jpg")])
prediction2= model.predict([prepare("C:\\Users\\Ganthier\\Desktop\\Mango\\mango_Fransik\\mangofransik_image-0001.jpg")])

def trouver_1(p):
    ar = []
    for n in p:
        ar.append(int(n))
    return ar.index(1, 0)

#ar.append(prediction[0].astype('int'))
#print(ar)
#print(ar.index(1, 0))
print(CATEGORIES[trouver_1(prediction[0])])
print(CATEGORIES[trouver_1(prediction1[0])])
print(CATEGORIES[trouver_1(prediction2[0])])


mango_corn
mango_fil
mango_corn
